In [35]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.2:
    #     return (999,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(count/total_records)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    random.seed(42)
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof



In [36]:
from sklearn.neighbors import KNeighborsClassifier

def bestIndividual_KNN(hof,df):
    k = 5
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/iono/ionosphere.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [37]:
from sklearn.model_selection import StratifiedKFold


def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=2000)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/iono/ionosphere.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [38]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier


def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/iono/ionosphere.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [39]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,1,2,3,4,5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=50
            n_gen=30
            # read dataframe from csv
            df = pd.read_csv('data/iono/ionosphere.csv')
            # encode labels column to numbers
            y = df['target']
            feature_columns = [ col for col in df.columns if col not in ['id','target','f2','f3']]
            for col in feature_columns:
                df[col] = pd.qcut(df[col], q=2, labels=False, duplicates='drop')
            # get accuracy with all features
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue


Accuracy with all features: 	(0.009375,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg      	min      
0  	50    	0.0360758	0.0100334
1  	29    	0.0212775	0.00986842
2  	30    	0.0156044	0.00986842
3  	30    	0.0139559	0.00986842
4  	29    	0.012221 	0.00986842
5  	35    	0.0109452	0.00986842
6  	21    	0.0104423	0.00986842
7  	26    	0.0100192	0.00980392
8  	30    	0.0101199	0.00980392
9  	27    	0.010018 	0.00967742
10 	28    	0.0101749	0.00967742
11 	29    	0.00989606	0.00967742
12 	33    	0.00980576	0.0096463 
13 	22    	0.00975712	0.0096463 
14 	30    	0.00975465	0.00958466
15 	31    	0.0100223 	0.00952381
16 	37    	0.00986201	0.00946372
17 	22    	0.00960245	0.00946372
18 	31    	0.00970882	0.00943396
19 	29    	0.00952299	0.00943396
20 	33    	0.00947688	0.00943396
21 	34    	0.00946178	0.00943396
22 	21    	0.0094521 	0.00943396
23 	33    	0.00951511	0.00943396
24 	29    	0.00944176	0.00943396
25 	25    	0.00946612	0.00943396
26 	30    	0.00951394	0.00943396
27 	30    	0.00953438	0.00943396
28 	32    	0.00946963	0.00943396
29 	29    	0.00946065	0.

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg     	min    
0  	50    	0.782059	0.67687
1  	29    	0.745347	0.614717
2  	30    	0.717141	0.525173
3  	30    	0.68491 	0.525173
4  	29    	0.636783	0.490028
5  	35    	0.597769	0.472456
6  	21    	0.546917	0.431771
7  	26    	0.524545	0.431771
8  	30    	0.492662	0.404274
9  	27    	0.468539	0.404274
10 	28    	0.452617	0.404274
11 	29    	0.439203	0.404274
12 	33    	0.420781	0.387577
13 	22    	0.409643	0.387577
14 	30    	0.411704	0.387577
15 	31    	0.419888	0.387577
16 	37    	0.409675	0.387577
17 	22    	0.398888	0.383237
18 	31    	0.406178	0.383237
19 	29    	0.399805	0.383237
20 	33    	0.394233	0.383237
21 	34    	0.391152	0.369149
22 	21    	0.394061	0.369149
23 	33    	0.395431	0.369149
24 	29    	0.386074	0.364349
25 	25    	0.39049 	0.364349
26 	30    	0.382688	0.364349
27 	30    	0.388681	0.341509
28 	32    	0.386197	0.341509
29 	29    	0.372241	0.341509
30 	21    	0.352459	0.339549
Top 1 solution: [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 

...

gen	nevals	avg    	min    
0  	50    	2.70325	2.20629
1  	29    	2.56376	2.20629
2  	30    	2.44797	1.69771
3  	30    	2.29259	1.63975
4  	29    	2.07064	1.06624
5  	35    	1.87809	1.20912
6  	21    	1.62754	1.06624
7  	26    	1.50885	1.03866
8  	30    	1.31647	0.843697
9  	27    	1.14255	0.79533 
10 	28    	1.08658	0.79533 
11 	29    	0.952569	0.642695
12 	33    	0.846319	0.665693
13 	22    	0.760118	0.665693
14 	30    	0.760791	0.550534
15 	31    	0.765414	0.474875
16 	37    	0.717783	0.474875
17 	22    	0.595196	0.447773
18 	31    	0.575319	0.447773
19 	29    	0.530299	0.447773
20 	33    	0.490724	0.447773
21 	34    	0.478863	0.447773
22 	21    	0.482399	0.447773
23 	33    	0.466234	0.447773
24 	29    	0.453211	0.447773
25 	25    	0.476315	0.447773
26 	30    	0.461668	0.447773
27 	30    	0.482128	0.447773
28 	32    	0.484429	0.447773
29 	29    	0.488874	0.447773
30 	21    	0.458759	0.447773
Top 1 solution: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	50    	3.44923	2.78464
1  	29    	3.26307	2.78464
2  	30    	3.10911	2.12221
3  	30    	2.9033 	2.04146
4  	29    	2.60853	1.25712
5  	35    	2.35404	1.47123
6  	21    	2.02184	1.25712
7  	26    	1.86275	1.23809
8  	30    	1.60371	0.960506
9  	27    	1.35367	0.903592
10 	28    	1.29808	0.730066
11 	29    	1.11006	0.574477
12 	33    	0.891309	0.574477
13 	22    	0.768692	0.574477
14 	30    	0.71758 	0.574477
15 	31    	0.701025	0.574477
16 	37    	0.684728	0.574477
17 	22    	0.621788	0.495586
18 	31    	0.667635	0.495586
19 	29    	0.646089	0.495586
20 	33    	0.611   	0.491969
21 	34    	0.594338	0.491969
22 	21    	0.572267	0.491969
23 	33    	0.536441	0.45632 
24 	29    	0.498364	0.45632 
25 	25    	0.534016	0.45632 
26 	30    	0.490492	0.45632 
27 	30    	0.501881	0.45632 
28 	32    	0.502036	0.45632 
29 	29    	0.507507	0.45632 
30 	21    	0.470212	0.45632 
Top 1 solution: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	50    	4.19521	3.36298
1  	29    	3.96239	3.36298
2  	30    	3.77025	2.54671
3  	30    	3.51401	2.44317
4  	29    	3.1464 	1.448  
5  	35    	2.83   	1.73334
6  	21    	2.41606	1.448  
7  	26    	2.21349	1.43753
8  	30    	1.88868	1.07731
9  	27    	1.57512	1.01185
10 	28    	1.52811	0.789895
11 	29    	1.28604	0.608665
12 	33    	1.01742	0.608665
13 	22    	0.848236	0.608665
14 	30    	0.779542	0.5985  
15 	31    	0.755071	0.5985  
16 	37    	0.748991	0.51268 
17 	22    	0.645372	0.51268 
18 	31    	0.649771	0.51268 
19 	29    	0.634361	0.51268 
20 	33    	0.577512	0.51268 
21 	34    	0.562582	0.51268 
22 	21    	0.570286	0.51268 
23 	33    	0.553466	0.51268 
24 	29    	0.523705	0.507934
25 	25    	0.581019	0.464867
26 	30    	0.542345	0.464867
27 	30    	0.566058	0.464867
28 	32    	0.549729	0.464867
29 	29    	0.530446	0.464867
30 	21    	0.483388	0.464867
Top 1 solution: [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [40]:
#seed 42 n_pop=50 n_gen=25 lamda=1
#['f15', 'f18', 'f21', 'f22', 'f25', 'f26', 'f33'] KNN=0.8775 SVM=0.9229 DT=0.869

#seed 42 n_pop=50 n_gen=25 lamda=0.9
#['f15', 'f18', 'f21', 'f22', 'f25', 'f26', 'f33'] KNN=0.8775 SVM=0.9229 DT=0.869

#seed 42 n_pop=50 n_gen=25 lamda=0.8
#['f9', 'f12', 'f18', 'f21', 'f22', 'f24', 'f25', 'f26', 'f33'] KNN=0.8376 SVM=0.9057 DT=0.8889

#seed 42 n_pop=50 n_gen=25 lamda=0.7
#['f9', 'f12', 'f18', 'f21', 'f22', 'f24', 'f25', 'f26', 'f33'] KNN=0.8376 SVM=0.9057 DT=0.8889

#seed 42 n_pop=50 n_gen=25 lamda=0.6
#['f9', 'f12', 'f18', 'f21', 'f22', 'f24', 'f25', 'f26', 'f33'] KNN=0.8376 SVM=0.9057 DT=0.8889

#seed 42 n_pop=50 n_gen=25 lamda=0.5
#['f2', 'f9', 'f14', 'f18', 'f21', 'f25', 'f26', 'f28', 'f33'] KNN=0.8719 SVM=0.9143 DT=0.8775

#seed 42 n_pop=50 n_gen=25 lamda=0.4
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f24', 'f25', 'f26', 'f28', 'f33'] KNN=0.8548 SVM=0.9058 DT=0.8976

#seed 42 n_pop=50 n_gen=25 lamda=0.3
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f24', 'f25', 'f26', 'f28', 'f33'] KNN=0.8548 SVM=0.9058 DT=0.8976

#seed 42 n_pop=50 n_gen=25 lamda=0.2
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f26', 'f28', 'f33'] KNN=0.8606 SVM=0.9087 DT=0.8749

#seed 42 n_pop=50 n_gen=25 lamda=0.1
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f26', 'f28', 'f33'] KNN=0.8606 SVM=0.9087 DT=0.8749

#seed 42 n_pop=50 n_gen=25 lamda=0
#['f2', 'f9', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f28', 'f32', 'f33'] KNN=0.8633 SVM=0.9287 DT=0.8663

In [41]:
def FN(condition_attrs, data, decision_attr):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data, condition_attrs)

    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if len(x)>total_records*0.01:
                y = 1-len(x) / class_size
                if y<0.3:
                    rule_confidences.append(y)
            if len(x)>0:
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)

    print(entropy_D)
    print(entropy_DB)
    print(rule_confidences)
    print(count)

    avg_confidence = np.sum(rule_confidences) / count +0*(entropy_D-entropy_DB)

    return avg_confidence

In [42]:
# df = pd.read_csv('../data/iono/ionosphere.csv')
# feature_columns = [ col for col in df.columns if col not in ['id','target','f2','f3']]
# for col in feature_columns:
#     df[col] = pd.qcut(df[col], q=2, labels=False, duplicates='drop')
#

In [43]:
# feature_columns=['f15', 'f18', 'f21', 'f22', 'f25', 'f26', 'f33']
# # feature_columns01=['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f26', 'f28', 'f33']
# print(FN(feature_columns,df,'target'))